### Import the libraries

In [1]:
import numpy as np
from numpy import nan
import math
import pandas as pd
from typing import List
from typing import Dict
from collections import defaultdict

In [2]:
#The name of the doc depends on what do you have it stored it as

df_students_original = pd.read_excel (r'./student_project_assignment_input-1.xls', sheet_name='studenten')
df_courses_original = pd.read_excel (r'./student_project_assignment_input-1.xls', sheet_name='assignments')


In [3]:
df_students = df_students_original.copy()

In [4]:
df_courses = df_courses_original.copy()

### Clean the data

In [5]:
#stablishing happiness points system

lst_1_8 = list(range(1,9))
lst_1_8.reverse()
lst_1_8
df_students.loc[0][2:] = lst_1_8

In [6]:
#cleaning (students)

df_students.columns = df_students.iloc[0]
df_students = df_students.drop(0)
df_students = df_students.reset_index(drop=True)

In [7]:
clean_empty_rows = df_students['Student ID'].isnull()
df_students_clean = df_students[~clean_empty_rows]

clean_empty_rows2 = df_students_clean['Credits Required'].isnull()
df_students_clean = df_students_clean[~clean_empty_rows2]
df_students_clean.reset_index(drop = True)
df_students_clean = df_students_clean.set_index(['Student ID'])

df_students = df_students_clean

In [8]:
#formatting (students)

credits_num = pd.to_numeric(df_students['Credits Required'])
df_students['Credits Required'] = credits_num

In [9]:
df_students.info()

<class 'pandas.core.frame.DataFrame'>
Index: 244 entries, s_3 to s_246
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Credits Required  244 non-null    int64 
 1   8                 244 non-null    object
 2   7                 244 non-null    object
 3   6                 244 non-null    object
 4   5                 244 non-null    object
 5   4                 244 non-null    object
 6   3                 244 non-null    object
 7   2                 244 non-null    object
 8   1                 244 non-null    object
dtypes: int64(1), object(8)
memory usage: 19.1+ KB


In [10]:
#cleaning (courses)

df_courses.columns = df_courses.iloc[0]
df_courses = df_courses.drop(0)
df_courses = df_courses.reset_index(drop=True)

In [11]:
clean_empty_rows3 = df_courses['Course ID'].isnull()
df_courses_clean = df_courses[~clean_empty_rows3]

df_courses_clean.reset_index(drop = True)
df_courses_clean = df_courses_clean.set_index(['Course ID'])

df_courses = df_courses_clean

In [12]:
#formatting (courses)


df_courses['Min Students'] = pd.to_numeric(df_courses['Min Students'])
df_courses['Max Students'] = pd.to_numeric(df_courses['Max Students'])
df_courses['Time Slot'] = pd.to_numeric(df_courses['Time Slot'])
df_courses['Credits'] = pd.to_numeric(df_courses['Credits'])

In [13]:
df_courses.info()

<class 'pandas.core.frame.DataFrame'>
Index: 46 entries, DA010 to DI703
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   Min Students  46 non-null     int64
 1   Max Students  46 non-null     int64
 2   Time Slot     46 non-null     int64
 3   Credits       46 non-null     int64
dtypes: int64(4)
memory usage: 1.8+ KB


### (Experiment) Order students trying to fill each timeslot first

In [14]:
#make a new dataframe ordered by timeslots

df_courses_timeslots = pd.DataFrame({})

for i in range(df_courses['Time Slot'].max()+1):
    col_name = 'Time Slot ' + str(i)
    df_new_column = pd.DataFrame({col_name : list(df_courses[df_courses['Time Slot'] == i].index)})
    df_courses_timeslots = pd.concat([df_courses_timeslots, df_new_column], axis=1)
    


In [15]:
#THERE MUST BE A MORE GENERAL WAY OF DOING THIS, FIX IN FUTURE.
#ACTUALLY, IGNORE THIS FOR NOW

df_timeslot_0 = df_courses_timeslots[['Time Slot 0']]
df_timeslot_1 = df_courses_timeslots[['Time Slot 1']]
df_timeslot_2 = df_courses_timeslots[['Time Slot 2']]
df_timeslot_3 = df_courses_timeslots[['Time Slot 3']]
df_timeslot_4 = df_courses_timeslots[['Time Slot 4']]
df_timeslot_5 = df_courses_timeslots[['Time Slot 5']]
df_timeslot_6 = df_courses_timeslots[['Time Slot 6']]
df_timeslot_9 = df_courses_timeslots[['Time Slot 9']]

#### Start of doing the artifitial preferences for each course

In [16]:
#Make a dictionary of the credits that the students still need, we will update this as we assign them to courses and
#take them out of the pool once they are full.

Credits_remaining = df_students[['Credits Required']].copy()
Credits_remaining = Credits_remaining.to_dict()
Credits_remaining 

{'Credits Required': {'s_3': 3,
  's_4': 3,
  's_5': 3,
  's_6': 3,
  's_7': 3,
  's_8': 3,
  's_9': 3,
  's_10': 3,
  's_11': 3,
  's_12': 3,
  's_13': 3,
  's_14': 4,
  's_15': 3,
  's_16': 3,
  's_17': 3,
  's_18': 3,
  's_19': 3,
  's_20': 3,
  's_21': 3,
  's_22': 3,
  's_23': 3,
  's_24': 4,
  's_25': 3,
  's_26': 3,
  's_27': 3,
  's_28': 3,
  's_29': 3,
  's_30': 3,
  's_31': 3,
  's_32': 3,
  's_33': 3,
  's_34': 3,
  's_35': 3,
  's_36': 3,
  's_37': 3,
  's_38': 3,
  's_39': 1,
  's_40': 4,
  's_41': 3,
  's_42': 3,
  's_43': 2,
  's_44': 3,
  's_45': 4,
  's_46': 3,
  's_47': 3,
  's_48': 3,
  's_49': 3,
  's_50': 3,
  's_51': 3,
  's_52': 3,
  's_53': 3,
  's_54': 3,
  's_55': 3,
  's_56': 3,
  's_57': 3,
  's_58': 3,
  's_59': 3,
  's_60': 3,
  's_61': 3,
  's_62': 3,
  's_63': 3,
  's_64': 3,
  's_65': 3,
  's_66': 4,
  's_67': 3,
  's_68': 3,
  's_69': 3,
  's_70': 3,
  's_71': 3,
  's_72': 3,
  's_73': 3,
  's_74': 3,
  's_75': 1,
  's_76': 3,
  's_77': 3,
  's_78': 3,

In [17]:
def order_students(students: List[str], course:str) -> List[str]:
    '''
    Takes in: 
    A list of students that want to sign up for a course
    The course
    The most recent version of credits_remaining
    
    Does:
    Orders a list of students based on these priorities:
    1-the happiness they would get for getting a course 
    2-how many credits they need
    '''
    
    #make a dataframe with the relevant stuff
    #    #select the students
    df_order = df_students.loc[students].copy()
    
    
    #    #make the happiness column
    Happy_list = []
    for i in df_order.index:
        Happy_list += [df_order.loc[i][df_order.loc[i]==course].index[0]]
    
    df_order = df_order.reset_index()
    coll = pd.DataFrame({'Happiness':Happy_list})
    df_order = pd.concat([df_order, coll], axis = 1)
    df_order = df_order.drop(['Credits Required',8,7,6,5,4,3,2,1], axis=1)
    df_order = df_order.set_index(['Student ID'])
    
    
    #    #include current credit necessities
    global Credits_remaining
    credit_col = { your_key: Credits_remaining['Credits Required'][your_key] for your_key in list(df_order.index) }
    credit_col = pd.Series(credit_col)
    credit_col = credit_col.to_frame()
    credit_col['Credits Required'] = credit_col[0]
    credit_col = credit_col.drop(0, axis=1)
    df_order = pd.concat([df_order, credit_col], axis = 1)
    
    #Order by priorities given
    df_order = df_order.sort_values(['Happiness', 'Credits Required'], ascending=[False, False])
    final_list = list(df_order.index)
    
    return final_list

In [18]:
#Create the students_per_course dataframe

students_per_course = pd.DataFrame({})
for course in df_courses.index:
    stud_list = []
    for stude in df_students.index:
        if course in list(df_students.loc[stude]):
            stud_list += [stude]
    stud_list = order_students(stud_list, course)
    col = pd.DataFrame({course:stud_list})
    students_per_course = pd.concat([students_per_course, col], axis = 1)
    

# Algorithm

## Make each Dataframe

In [20]:
#Here we create the dataframes of students per each timeslot
#We use a function for convenience

df_students_onlycourses = df_students[[1,2,3,4,5,6,7,8]].copy()

def timeslotter( tslot:int) -> pd.DataFrame:
    ''' Creates a dataframe of the courses only in a particular timeslot that students are taking, in order of preference'''
    
    courses_in_timeslot = set(df_courses_timeslots['Time Slot ' + str(tslot)])-{nan}
    students_ind = set()
    
    for course in courses_in_timeslot:
        students_ind = students_ind | set(students_per_course[course])

    students_ind = list(students_ind -{nan})

    name = pd.DataFrame(columns=['student', 'index'])
    name['student'] = students_ind

    for i in range(len(name)):
        name['index'][i] = int(name.loc[i]['student'][2:])
    name = name.sort_values('index')
    name = name.set_index('student')
    name = name.drop('index', axis=1)

    name['Courses'] = ''

    all_crses=[]
    
    for stud in name.index:
        crses = []
        for cours in df_students_onlycourses.loc[stud]:
            if (cours in courses_in_timeslot) and not(cours in crses):
                crses = [cours] + crses
        all_crses += [crses]
    name['Courses'] = all_crses
    
    return name

In [21]:
#Now we use the function to make all the dataframes, scuffed method, but for now it will suffice. 
#I could do this with a loop but im tired

df_students_timeslot_0 = timeslotter(0)
df_students_timeslot_1 = timeslotter(1)
df_students_timeslot_2 = timeslotter(2)
df_students_timeslot_3 = timeslotter(3)
df_students_timeslot_4 = timeslotter(4)
df_students_timeslot_5 = timeslotter(5)
df_students_timeslot_6 = timeslotter(6)
df_students_timeslot_9 = timeslotter(9)

timeslotsss = [df_students_timeslot_0, df_students_timeslot_1, df_students_timeslot_2, df_students_timeslot_3, df_students_timeslot_4, df_students_timeslot_5, df_students_timeslot_6, df_students_timeslot_9]

## Some functions 

In [22]:
#Auxiliary variables

def get_max(course:'str')-> int:
    return df_courses['Max Students'][course]

def get_min(course:'str')-> int:
    return df_courses['Min Students'][course]

def get_cred(course:'str')-> int:
    return df_courses['Credits'][course]

def get_pref(dframe : pd.DataFrame, student:str, course:str) -> int:
    return dframe.loc[student]['Courses'].index(course)

## Pieces of the algorithm

In [23]:
#A list of all the students with all their courses filled already
#We will check this list before assigning students to new courses
full_students = [] 

In [24]:
def start_algorithm(dframe: pd.DataFrame, tslot:int)-> [Dict[str, List[str]], pd.DataFrame]:
    '''Executes 1st step of the deferred acceptance algorithm for a specific timeslot
    It needs the df_students_timeslot_x and the timeslot as inputs.
    It outputs both the matching and the updated dataframe.
    '''
    
    global Credits_remaining 
    global full_students

    #we will update this in order to keep track of who is full, and possibly use that in the algorithm. 
    #For now, I will not do this yet
    
    #Make a dictionary where we will assign the students
    courses_in_timeslot = list(set(df_courses_timeslots['Time Slot ' + str(tslot)])-{nan})
    
    Matching = defaultdict()
    for i in courses_in_timeslot:
        Matching[i] = []
        
    
    #Step 1: assign everyone to preference 1
    for stud in dframe.index:
        if not(stud in full_students): 
            dest = dframe.loc[stud]['Courses'][0]
            Matching[dest] += [stud]
        
    #Step 2: improve the matching by the deferred acceptance algorithm
    for crs in Matching.keys():
        Matching[crs] = order_students(Matching[crs], crs)
    
    #    #Now we cull and do 1 round of downgrading preferences.
    
    for crs in Matching.keys():
        if len(Matching[crs]) > get_max(crs):
            lst_of_students = Matching[crs]
            Matching[crs] = lst_of_students[:get_max(crs)]
            rejected_students = lst_of_students[get_max(crs):]
            
            for stu in rejected_students:
                pref =  get_pref(dframe, stu, crs) + 1
                
                if pref < len(dframe.loc[stu]['Courses']):
                    new_dest = dframe.loc[stu]['Courses'][pref]
                    Matching[new_dest] = Matching[new_dest] + [stu]
    #1 round has been completed.
    
    #update credits
    for crse in Matching.keys():
        credits = get_cred(crse)
        for student in Matching[crse]:
            Credits_remaining['Credits Required'][student] -= credits
    
    #Update full students list
    for student in Credits_remaining['Credits Required']:
        if Credits_remaining['Credits Required'][student] <= 0:
            full_students += [student]
        
    return [Matching, dframe]

In [25]:
def turn_into_df(Matching:Dict[str, List[str]]) -> pd.DataFrame:
    ''' turns a dict into a dataframe
    '''
    df_match = pd.DataFrame()
    for i in Matching.keys():
        col = pd.DataFrame({i:Matching[i]})
        df_match = pd.concat([df_match, col], axis=1)
    return df_match

In [26]:
def next_step(Matching : Dict[str, List[str]], dframe: pd.DataFrame) -> [Dict[str, List[str]], pd.DataFrame]:
    '''Does the next step of the algorithm for a given matching of a given Dataframe
    '''
    global Credits_remaining
    
    #Order
    for crs in Matching.keys():
        Matching[crs] = order_students(Matching[crs], crs)
    
    #Cull last ones
    for crs in Matching.keys():
        credits = get_cred(crs)
        if len(Matching[crs]) > get_max(crs):
            lst_of_students = Matching[crs]
            Matching[crs] = lst_of_students[:get_max(crs)]
            rejected_students = lst_of_students[get_max(crs):]
            
            for stu in rejected_students:
                pref =  get_pref(dframe, stu, crs) + 1
                
                if pref < len(dframe.loc[stu]['Courses']):
                    new_dest = dframe.loc[stu]['Courses'][pref]
                    Matching[new_dest] = Matching[new_dest] + [stu]
                else:
                    Credits_remaining['Credits Required'][stu] += credits
                
        
    return [Matching, dframe]

In [27]:
def delete_courses(Matching : Dict[str, List[str]], dframe: pd.DataFrame) -> [Dict[str, List[str]], pd.DataFrame]:
    '''Checks a completed matching for a given dataframe and deletes courses that dont have the min. students required,
    while taking care of updating credits and moving students and such.
    '''
    global Credits_remaining
    removings = []
    
    for course in Matching.keys():
        if len(Matching[course]) < get_min(course):
            rejected_students = Matching[course]
            Matching[course] = []
            
            credits = get_cred(course)
            
            removings += [course]
            
            for stu in rejected_students:
                pref =  get_pref(dframe, stu, course) + 1
                
                if pref < len(dframe.loc[stu]['Courses']):
                    new_dest = dframe.loc[stu]['Courses'][pref]
                    Matching[new_dest] = Matching[new_dest] + [stu]
                else:
                    Credits_remaining['Credits Required'][stu] += credits 
    
    for cou in removings:
        if Matching[cou] == []:
            del Matching[cou]
                    
    return [Matching, dframe]

# Algorithm 1

In [28]:
## Fill each timeslot first, 

def algorithm1()-> List[Dict[str, List[str]]]:
    '''An algorithm that fills each timeslot first, then moves on to the next timeslot
    '''
    list_all = []    #We will store the dicts here
    
    for dframe in timeslotsss:    #loops over all the timeslots
        
        tslot = df_courses.loc[dframe.iloc[0][0][0]]['Time Slot']
        
        ans_old = start_algorithm(dframe, tslot)
        ans = next_step(ans_old[0], ans_old[1])
        while ans[0] != ans_old[0]:
            ans_old = ans
            ans = next_step(ans_old[0], ans_old[1])
            
#    #THE DELETE -> SHUFFLE PROCESS HAS TO BE A LOOP ITSELF, FIX NEEDED.

        ans_old = delete_courses(ans[0], ans[1])
        ans = next_step(ans_old[0], ans_old[1])
        while ans[0] != ans_old[0]:
            ans_old = ans
            ans = next_step(ans_old[0], ans_old[1])
        
        list_all = list_all + [ans]
    return list_all

In [29]:
a = algorithm1()

In [30]:
#solution for now:
sol = []
for i in a:
    sol += [i[0]]
    print(turn_into_df(i[0]))

    DA011
0   s_223
1    s_90
2   s_139
3    s_68
4    s_78
5   s_216
6    s_15
7    s_22
8    s_38
9   s_107
10  s_135
11  s_176
12   s_82
13  s_127
14   s_72
15   s_83
16   s_96
17  s_126
18  s_108
19  s_189
    DA912  DA206  DA512  DA201  DA718
0     s_6  s_128   s_20  s_157   s_46
1    s_21  s_126   s_25   s_10  s_165
2    s_60  s_155   s_30   s_87   s_24
3    s_64  s_170  s_118  s_109   s_50
4   s_105  s_206  s_172  s_158  s_101
5   s_144   s_76  s_236  s_167  s_119
6   s_145  s_122  s_217  s_178  s_177
7   s_175   s_39   s_66  s_185  s_191
8   s_188    s_7    s_3  s_193  s_215
9   s_229   s_26    s_8   s_82  s_219
10   s_96   s_55   s_19   s_29  s_110
11   s_75  s_114   s_48   s_56  s_135
12  s_131  s_151   s_80   s_58  s_213
13   s_95   s_43  s_104   s_70   s_28
14    s_4   s_93    NaN   s_92   s_32
15    NaN  s_148    NaN  s_113   s_91
16    NaN  s_154    NaN  s_146  s_198
17    NaN  s_211    NaN  s_179   s_45
18    NaN  s_227    NaN  s_197   s_71
19    NaN  s_173    NaN  s_205

In [31]:
sol

[defaultdict(None,
             {'DA011': ['s_223',
               's_90',
               's_139',
               's_68',
               's_78',
               's_216',
               's_15',
               's_22',
               's_38',
               's_107',
               's_135',
               's_176',
               's_82',
               's_127',
               's_72',
               's_83',
               's_96',
               's_126',
               's_108',
               's_189']}),
 defaultdict(None,
             {'DA912': ['s_6',
               's_21',
               's_60',
               's_64',
               's_105',
               's_144',
               's_145',
               's_175',
               's_188',
               's_229',
               's_96',
               's_75',
               's_131',
               's_95',
               's_4'],
              'DA206': ['s_128',
               's_126',
               's_155',
               's_170',
               

In [32]:
ddd = defaultdict()
for stu in df_students.index:
    ddd[stu] = []

for tslot in sol:
    for course in tslot:
        for student in tslot[course]:
            ddd[student] = ddd[student] + [course]
            
ddd

defaultdict(None,
            {'s_3': ['DA512', 'DA707', 'DA302'],
             's_4': ['DA912', 'DA605', 'DA212'],
             's_5': ['DA217', 'DA803'],
             's_6': ['DA912', 'DA803'],
             's_7': ['DA206', 'DA910-2', 'DA251F'],
             's_8': ['DA512', 'DA301-1', 'DA408'],
             's_9': ['DA302', 'DA104'],
             's_10': ['DA201', 'DA714'],
             's_11': ['DA212', 'DA104'],
             's_12': ['DA207', 'DA605'],
             's_13': ['DA803', 'DA212'],
             's_14': ['DA250F-2', 'DA302', 'DA311'],
             's_15': ['DA011', 'DA910-1', 'DA717'],
             's_16': ['DA511', 'DA717', 'DA302'],
             's_17': ['DA910-1', 'DA701', 'DA311'],
             's_18': ['DA910-2', 'DA207'],
             's_19': ['DA512', 'DA301-1', 'DA806'],
             's_20': ['DA512', 'DA714', 'DA204'],
             's_21': ['DA912', 'DA719-1', 'DA806'],
             's_22': ['DA011', 'DA511', 'DA806'],
             's_23': ['DA217', 'DA511', 'DA

### --------IGNORE

In [33]:
#data = {'DA010': ['s_60', 's_132', 's_145', 's_150', 's_153', 's_183', 's_196', 's_243'], 'DA011': ['s_68', 's_90', 's_139', 's_216', 's_223'], 'DA011F': ['s_85', 's_222'], 'DA104': ['s_5', 's_7', 's_8', 's_9', 's_11', 's_12', 's_38', 's_40', 's_42', 's_70', 's_74', 's_84', 's_90', 's_98', 's_102', 's_109', 's_114', 's_125', 's_126', 's_143', 's_151', 's_162', 's_164', 's_165', 's_182', 's_184', 's_191', 's_195', 's_200', 's_202', 's_203', 's_208', 's_211', 's_231', 's_244', 's_246'], 'DA201': ['s_9', 's_10', 's_29', 's_56', 's_58', 's_70', 's_82', 's_87', 's_92', 's_94', 's_109', 's_113', 's_120', 's_128', 's_146', 's_147', 's_156', 's_157', 's_158', 's_167', 's_178', 's_179', 's_185', 's_192', 's_193', 's_197', 's_202', 's_205', 's_207', 's_209', 's_212', 's_240'], 'DA202': ['s_84', 's_137', 's_235', 's_236'], 'DA204': ['s_124', 's_160'], 'DA206': ['s_25', 's_126', 's_128', 's_155', 's_170', 's_246'], 'DA207': ['s_12', 's_76', 's_79', 's_93', 's_104', 's_121', 's_136', 's_138', 's_173', 's_174', 's_177', 's_215', 's_218', 's_220', 's_228', 's_242'], 'DA211': ['s_72', 's_128'], 'DA212': ['s_4', 's_7', 's_11', 's_13', 's_21', 's_38', 's_47', 's_57', 's_59', 's_68', 's_81', 's_87', 's_95', 's_98', 's_100', 's_110', 's_124', 's_126', 's_135', 's_142', 's_144', 's_145', 's_148', 's_174', 's_176', 's_180', 's_185', 's_188', 's_204', 's_209', 's_216', 's_227', 's_230', 's_237'], 'DA217': ['s_24', 's_43', 's_51', 's_57', 's_61', 's_152', 's_153', 's_177', 's_180', 's_182', 's_201', 's_203', 's_230', 's_237', 's_241'], 'DA220': [], 'DA234': ['s_27', 's_28', 's_184', 's_198', 's_222', 's_223', 's_225'], 'DA238': ['s_105', 's_184'], 'DA250F-1': ['s_14', 's_181'], 'DA250F-2': ['s_173', 's_176'], 'DA251F': ['s_24', 's_47', 's_183', 's_236'], 'DA301-1': ['s_6', 's_8', 's_33', 's_34', 's_64', 's_66', 's_83', 's_129', 's_130', 's_146', 's_155', 's_161', 's_162', 's_163', 's_178', 's_190', 's_195', 's_201', 's_237'], 'DA302': ['s_3', 's_18', 's_36', 's_44', 's_89', 's_151', 's_186', 's_195', 's_227', 's_243'], 'DA306': ['s_27', 's_28', 's_112', 's_150', 's_157', 's_168', 's_176', 's_187', 's_204'], 'DA308': ['s_26', 's_86', 's_133', 's_158', 's_213'], 'DA311': ['s_11', 's_17', 's_32', 's_34', 's_40', 's_41', 's_55', 's_77', 's_83', 's_89', 's_107', 's_123', 's_127', 's_141', 's_150', 's_154', 's_159', 's_168', 's_170', 's_193', 's_198', 's_206', 's_224', 's_229', 's_239', 's_244'], 'DA403': ['s_33', 's_48', 's_102', 's_108', 's_112', 's_135', 's_172', 's_239'], 'DA408': ['s_55', 's_60', 's_160', 's_216', 's_234', 's_242', 's_245'], 'DA415F': ['s_14', 's_19', 's_20', 's_22', 's_36', 's_41', 's_54', 's_57', 's_61', 's_65', 's_72', 's_86', 's_96', 's_97', 's_99', 's_102', 's_108', 's_111', 's_114', 's_118', 's_121', 's_137', 's_149', 's_162', 's_166', 's_169', 's_192', 's_198', 's_201', 's_215', 's_221', 's_222', 's_231', 's_232', 's_240'], 'DA511': ['s_16', 's_22', 's_23', 's_42', 's_45', 's_89', 's_90', 's_95', 's_108', 's_124', 's_127', 's_139', 's_155', 's_161', 's_163', 's_164', 's_175', 's_187', 's_196', 's_202', 's_230', 's_234'], 'DA512': ['s_3', 's_4', 's_8', 's_14', 's_15', 's_19', 's_20', 's_25', 's_30', 's_48', 's_66', 's_80', 's_82', 's_88', 's_89', 's_99', 's_101', 's_104', 's_118', 's_120', 's_122', 's_127', 's_129', 's_137', 's_159', 's_161', 's_167', 's_172', 's_217', 's_220', 's_235', 's_236', 's_238'], 'DA513': ['s_16', 's_45', 's_96', 's_100', 's_106', 's_142', 's_148', 's_149', 's_166', 's_196', 's_240'], 'DA605': ['s_9', 's_12', 's_23', 's_29', 's_30', 's_33', 's_41', 's_45', 's_51', 's_68', 's_74', 's_88', 's_91', 's_100', 's_114', 's_125', 's_151', 's_152', 's_166', 's_204', 's_211', 's_214', 's_232', 's_238', 's_244'], 'DA701': ['s_17', 's_26', 's_44', 's_54', 's_58', 's_67', 's_73', 's_81', 's_93', 's_94', 's_103', 's_116', 's_119', 's_136', 's_181', 's_186', 's_194', 's_199', 's_208', 's_228', 's_241'], 'DA702': ['s_81', 's_88', 's_116', 's_120', 's_130', 's_152', 's_171', 's_181', 's_226'], 'DA704': ['s_27', 's_28', 's_35', 's_37', 's_49', 's_53', 's_67', 's_69', 's_73', 's_78', 's_111', 's_140', 's_171', 's_182', 's_189', 's_199', 's_200', 's_206', 's_210', 's_233'], 'DA707': ['s_3', 's_14', 's_18', 's_23', 's_31', 's_35', 's_36', 's_37', 's_40', 's_44', 's_49', 's_52', 's_53', 's_59', 's_62', 's_63', 's_65', 's_66', 's_73', 's_77', 's_78', 's_92', 's_93', 's_94', 's_97', 's_99', 's_101', 's_103', 's_104', 's_107', 's_111', 's_112', 's_116', 's_117', 's_122', 's_130', 's_134', 's_136', 's_140', 's_141', 's_147', 's_149', 's_156', 's_159', 's_163', 's_168', 's_170', 's_179', 's_186', 's_194', 's_210', 's_212', 's_218', 's_226', 's_233', 's_238', 's_241'], 'DA714': ['s_10', 's_20', 's_22', 's_24', 's_31', 's_32', 's_34', 's_40', 's_48', 's_50', 's_55', 's_59', 's_61', 's_62', 's_72', 's_91', 's_97', 's_103', 's_105', 's_117', 's_132', 's_154', 's_180', 's_210', 's_231'], 'DA717': ['s_15', 's_16', 's_43', 's_56', 's_79', 's_91', 's_106', 's_117', 's_132', 's_138', 's_139', 's_143', 's_148', 's_160', 's_169', 's_188', 's_191', 's_192', 's_205', 's_222', 's_224', 's_227', 's_229', 's_231', 's_234', 's_245'], 'DA718': ['s_24', 's_46', 's_50', 's_101', 's_110', 's_118', 's_119', 's_135', 's_165', 's_177', 's_191', 's_215', 's_219'], 'DA719-1': ['s_21', 's_39', 's_51', 's_80', 's_115', 's_154', 's_169', 's_175'], 'DA719-2': ['s_115'], 'DA803': ['s_5', 's_6', 's_13', 's_38', 's_54', 's_71', 's_123', 's_144', 's_153', 's_190', 's_219'], 'DA806': ['s_19', 's_46', 's_64', 's_85', 's_108', 's_115', 's_123', 's_157', 's_164', 's_165', 's_211', 's_221', 's_243'], 'DA910-1': ['s_15', 's_17', 's_46', 's_47', 's_50', 's_52', 's_63', 's_77', 's_106', 's_107', 's_113', 's_119', 's_121', 's_134', 's_147', 's_172', 's_183', 's_194', 's_203', 's_214', 's_235', 's_239'], 'DA910-2': ['s_7', 's_18', 's_31', 's_32', 's_42', 's_52', 's_53', 's_62', 's_65', 's_69', 's_76', 's_80', 's_83', 's_84', 's_122', 's_134', 's_141', 's_142', 's_143', 's_156', 's_208', 's_213', 's_214', 's_219', 's_221', 's_226', 's_233'], 'DA912': ['s_4', 's_6', 's_21', 's_25', 's_45', 's_60', 's_63', 's_64', 's_74', 's_75', 's_85', 's_95', 's_96', 's_98', 's_105', 's_125', 's_131', 's_144', 's_145', 's_175', 's_188', 's_190', 's_223', 's_224', 's_225', 's_229', 's_246'], 'DI702': [], 'DI703': ['s_30', 's_66', 's_71', 's_95', 's_129', 's_189', 's_197', 's_207']}

In [34]:

#key_list = list(data)

In [35]:
 #pd.set_option("display.max_rows", None, "display.max_columns", None)

### Keep track of changes/DataFrames

In [36]:
df_students

,Credits Required,8,7,6,5,4,3,2,1
Student ID,,,,,,,,,
s_3,3,DA707,DA512,DA302,DA702,DA714,DA415F,DA910-2,DA311
s_4,3,DA212,DA912,DA512,DA605,DA714,DA302,DA104,DA311
s_5,3,DA104,DA803,DA217,DA306,DA311,DA212,DA806,DA513
s_6,3,DA912,DA301-1,DA803,DA717,DA104,DA513,DA217,DA605
s_7,3,DA910-2,DA212,DA104,DA302,DA311,DA206,DA251F,DA803
...,...,...,...,...,...,...,...,...,...
s_242,3,DA207,DA408,DA912,DA714,DA803,DA717,DA719-1,DA415F
s_243,3,DA806,DA010,DA302,DA605,DA217,DA204,DA251F,DA415F
s_244,3,DA104,DA311,DA605,DA717,DA201,DA511,DA302,DA206


In [37]:
df_courses

,Min Students,Max Students,Time Slot,Credits
Course ID,,,,
DA010,5,20,2,1
DA011,5,20,0,1
DA011F,10,24,3,1
DA104,5,20,9,1
DA201,5,20,1,2
DA202,5,20,5,1
DA204,4,60,9,1
DA206,12,20,1,1
DA207,5,20,3,2


In [38]:
df_courses_timeslots

,Time Slot 0,Time Slot 1,Time Slot 2,Time Slot 3,Time Slot 4,Time Slot 5,Time Slot 6,Time Slot 7,Time Slot 8,Time Slot 9
0,DA011,DA201,DA010,DA011F,DA707,DA202,DA212,NaN,NaN,DA104
1,NaN,DA206,DA217,DA207,DA803,DA211,DA302,NaN,NaN,DA204
2,NaN,DA250F-1,DA250F-2,DA220,NaN,DA238,DA308,NaN,NaN,DA311
3,NaN,DA512,DA704,DA234,NaN,DA301-1,DA408,NaN,NaN,DA403
4,NaN,DA718,DA910-2,DA251F,NaN,DA306,DA701,NaN,NaN,DA415F
5,NaN,DA912,NaN,DA511,NaN,DA605,DA719-2,NaN,NaN,DA513
6,NaN,NaN,NaN,DA714,NaN,DA717,DA806,NaN,NaN,DA702
7,NaN,NaN,NaN,DA910-1,NaN,DA719-1,NaN,NaN,NaN,DI703
8,NaN,NaN,NaN,NaN,NaN,DI702,NaN,NaN,NaN,NaN


In [39]:
students_per_course

,DA010,DA011,DA011F,DA104,DA201,DA202,DA204,DA206,DA207,DA211,...,DA718,DA719-1,DA719-2,DA803,DA806,DA910-1,DA910-2,DA912,DI702,DI703
0,s_153,s_223,s_222,s_5,s_157,s_84,s_160,s_128,s_76,s_72,...,s_46,s_39,s_115,s_71,s_19,s_15,s_7,s_6,s_118,s_66
1,s_60,s_90,s_29,s_8,s_10,s_236,s_124,s_126,s_138,s_128,...,s_165,s_21,s_74,s_5,s_85,s_50,s_18,s_21,s_193,s_129
2,s_132,s_139,s_85,s_9,s_82,s_137,s_20,s_25,s_173,s_70,...,s_24,s_115,s_229,s_13,s_115,s_52,s_32,s_60,s_50,s_189
3,s_243,s_68,s_84,s_12,s_87,s_235,s_151,s_155,s_220,s_109,...,s_50,s_154,s_26,s_6,s_243,s_107,s_42,s_64,s_77,s_197
4,s_86,s_78,s_172,s_38,s_109,s_76,s_162,s_170,s_228,s_196,...,s_101,s_175,s_175,s_38,s_64,s_113,s_65,s_96,s_80,s_207
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,NaN,NaN,NaN,s_204,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
116,NaN,NaN,NaN,s_205,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
117,NaN,NaN,NaN,s_228,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
118,NaN,NaN,NaN,s_229,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
Credits_remaining

{'Credits Required': {'s_3': 0,
  's_4': 0,
  's_5': 0,
  's_6': 0,
  's_7': 0,
  's_8': 0,
  's_9': 1,
  's_10': 0,
  's_11': 1,
  's_12': 0,
  's_13': 0,
  's_14': 1,
  's_15': 0,
  's_16': 0,
  's_17': 0,
  's_18': 0,
  's_19': 0,
  's_20': 0,
  's_21': 0,
  's_22': 0,
  's_23': 0,
  's_24': 0,
  's_25': 1,
  's_26': 0,
  's_27': 0,
  's_28': 0,
  's_29': 0,
  's_30': 0,
  's_31': 0,
  's_32': 0,
  's_33': 0,
  's_34': 0,
  's_35': 0,
  's_36': 1,
  's_37': 1,
  's_38': -1,
  's_39': 0,
  's_40': 1,
  's_41': 0,
  's_42': 0,
  's_43': 0,
  's_44': 0,
  's_45': 0,
  's_46': 0,
  's_47': 0,
  's_48': 0,
  's_49': 1,
  's_50': 1,
  's_51': 0,
  's_52': 0,
  's_53': 0,
  's_54': 0,
  's_55': 0,
  's_56': 1,
  's_57': 0,
  's_58': 0,
  's_59': 0,
  's_60': 0,
  's_61': 1,
  's_62': 0,
  's_63': 0,
  's_64': 0,
  's_65': 1,
  's_66': 0,
  's_67': 1,
  's_68': 0,
  's_69': 0,
  's_70': 0,
  's_71': 0,
  's_72': 0,
  's_73': 0,
  's_74': 0,
  's_75': 0,
  's_76': -1,
  's_77': 1,
  's_78': 

In [41]:
timeslotsss


[         Courses
 student         
 s_15     [DA011]
 s_22     [DA011]
 s_38     [DA011]
 s_68     [DA011]
 s_72     [DA011]
 s_78     [DA011]
 s_82     [DA011]
 s_83     [DA011]
 s_90     [DA011]
 s_96     [DA011]
 s_107    [DA011]
 s_108    [DA011]
 s_126    [DA011]
 s_127    [DA011]
 s_135    [DA011]
 s_139    [DA011]
 s_176    [DA011]
 s_189    [DA011]
 s_216    [DA011]
 s_223    [DA011],
                        Courses
 student                       
 s_3                    [DA512]
 s_4             [DA912, DA512]
 s_6                    [DA912]
 s_7                    [DA206]
 s_8             [DA512, DA718]
 ...                        ...
 s_240           [DA201, DA512]
 s_242                  [DA912]
 s_244           [DA201, DA206]
 s_245                  [DA912]
 s_246    [DA912, DA206, DA512]
 
 [185 rows x 1 columns],
                 Courses
 student                
 s_3           [DA910-2]
 s_5             [DA217]
 s_6             [DA217]
 s_7           [DA910-2]
 s_10     

In [42]:
full_students

['s_39',
 's_75',
 's_82',
 's_131',
 's_217',
 's_28',
 's_39',
 's_43',
 's_75',
 's_78',
 's_82',
 's_92',
 's_109',
 's_131',
 's_146',
 's_178',
 's_189',
 's_197',
 's_206',
 's_217',
 's_7',
 's_10',
 's_18',
 's_27',
 's_28',
 's_29',
 's_32',
 's_37',
 's_39',
 's_43',
 's_49',
 's_56',
 's_60',
 's_67',
 's_70',
 's_75',
 's_76',
 's_78',
 's_82',
 's_92',
 's_93',
 's_104',
 's_109',
 's_110',
 's_111',
 's_113',
 's_122',
 's_127',
 's_131',
 's_138',
 's_140',
 's_145',
 's_146',
 's_173',
 's_177',
 's_178',
 's_189',
 's_197',
 's_205',
 's_206',
 's_210',
 's_211',
 's_213',
 's_215',
 's_217',
 's_221',
 's_5',
 's_6',
 's_7',
 's_10',
 's_18',
 's_27',
 's_28',
 's_29',
 's_32',
 's_38',
 's_39',
 's_43',
 's_60',
 's_70',
 's_71',
 's_75',
 's_76',
 's_78',
 's_82',
 's_92',
 's_93',
 's_102',
 's_104',
 's_109',
 's_110',
 's_111',
 's_113',
 's_115',
 's_122',
 's_127',
 's_131',
 's_138',
 's_140',
 's_144',
 's_145',
 's_146',
 's_153',
 's_173',
 's_177',
 's_17